<a href="https://colab.research.google.com/github/ismaelvacco/eleicoes/blob/main/city_with_issues_in_the_pop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 47 kB/s 
     |████████████████████████████████| 199 kB 51.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=6f2feb91e31fc2fba8e414ce8531f34cc7fa40e241a7a38151f38459663dceff
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pandas as pd
import numpy as np
import string
import seaborn as sns

from matplotlib import pyplot as plt
from google.colab import drive


In [ ]:
spark = SparkSession.builder.appName("VotosEUrnas").getOrCreate()

pd.options.display.html.table_schema=True
path = "/content/drive"
drive.mount(path)
votacao = spark.read.parquet(path + "/MyDrive/vc")

accent_replacements_spanish = [
    (u'á', 'a'), (u'Á', 'A'),
    (u'â', 'a'), (u'Â', 'A'),
    (u'ã', 'a'), (u'Ã', 'A'),
    (u'é', 'e'), (u'É', 'E'),
    (u'ê', 'e'), (u'Ê', 'E'),
    (u'í', 'i'), (u'Í', 'I'),
    (u'ò', 'o'), (u'Ó', 'O'),
    (u'ô', 'o'), (u'Ô', 'O'),
    (u'õ', 'o'), (u'Õ', 'O'),
    (u'ú|ü', 'u'), (u'Ú|Ű', 'U'),
    (u'ñ', 'n'),
    (u'ç', 'c'), (u'Ç', 'C'),
    # see http://stackoverflow.com/a/18123985/3810493 for other characters

    # this will convert other non ASCII characters to a question mark:
    ('[^\x00-\x7F]', '?') 
]

def remove_accents(column):
    r = col(column)
    for a, b in accent_replacements_spanish:
        r = regexp_replace(r, a, b)
    return r

votacao = votacao.withColumn('city_no_accents', remove_accents('cidade'))
votacao.createOrReplaceTempView("vc")

municipios = spark.read.csv(path + "/MyDrive/municipios", header=True)
municipios.createOrReplaceTempView("municipios")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
votacao.printSchema()

root
 |-- bolsonaro: integer (nullable = true)
 |-- lula: integer (nullable = true)
 |-- tebet: integer (nullable = true)
 |-- ciro: integer (nullable = true)
 |-- soraya: integer (nullable = true)
 |-- davila: integer (nullable = true)
 |-- kelmon: integer (nullable = true)
 |-- pericles: integer (nullable = true)
 |-- sofia: integer (nullable = true)
 |-- vera: integer (nullable = true)
 |-- eymael: integer (nullable = true)
 |-- brancos: integer (nullable = true)
 |-- nulos: integer (nullable = true)
 |-- cidade: string (nullable = true)
 |-- uf: string (nullable = true)
 |-- zona: integer (nullable = true)
 |-- secao: integer (nullable = true)
 |-- local_votacao: integer (nullable = true)
 |-- urna_id1: integer (nullable = true)
 |-- urna_id2: integer (nullable = true)
 |-- tipo_urna1: string (nullable = true)
 |-- tipo_urna2: string (nullable = true)
 |-- data_geracao: string (nullable = true)
 |-- hora_geracao: string (nullable = true)
 |-- data_abertura: string (nullable = true)

In [ ]:
# sql = "SELECT regiao, cidade, count(*) as qty_urnas, sum(abstencoes)/sum(aptos) * 100 as perc_abst FROM vc GROUP BY 1, 2 HAVING count(*) > 100 order by perc_abst desc"

sql = '''
  SELECT 
    cidade, 
    MAX(A.uf) as uf, 
    MAX(B.pop_21) as pop, 
    SUM(A.aptos) as aptos,
    ((SUM(A.aptos)/MAX(B.pop_21)) -1) * 100 as error,
    CASE WHEN sum(bolsonaro) > sum(lula) THEN 'BOLSONARO' 
      ELSE 'LULA' END as winner 
  FROM vc A 
    JOIN municipios B ON lower(A.city_no_accents)=lower(B.no_accents) and lower(A.uf)=lower(B.uf_code) 
    GROUP BY cidade HAVING SUM(A.aptos)>(MAX(B.pop_21))'''
city_with_issues = spark.sql(sql)
city_with_issues.createOrReplaceTempView("city_issues")

sql = '''
    select cidade, uf, round(error, 1) as error, aptos, pop, winner from city_issues order by error desc
'''
pdf = spark.sql(sql).toPandas()
pdf



,cidade,uf,error,aptos,pop,winner
0,BELÉM,PB,25559.1,1084354,4226,LULA
1,CAMPO GRANDE,MS,6685.1,649745,9576,BOLSONARO
2,CACHOEIRINHA,TO,5229.0,122194,2293,BOLSONARO
3,RIO BRANCO,MT,5161.1,270789,5147,BOLSONARO
4,VÁRZEA GRANDE,PI,4071.0,182772,4382,BOLSONARO
...,...,...,...,...,...,...
687,SÃO MIGUEL DO ALEIXO,SE,0.2,3972,3964,LULA
688,ITUTINGA,MG,0.1,3752,3749,LULA
689,PIEDADE DOS GERAIS,MG,0.1,5040,5034,LULA
690,SANTA TEREZA,RS,0.1,1724,1722,BOLSONARO
